# 1.Setup

In [4]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tempfile

# 2.构建 Keras 模型

In [2]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(16, activation = 'relu', input_shape = (4,)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(3)
])
model.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    optimizer = 'adam'
)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                80        
_________________________________________________________________
dropout (Dropout)            (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 51        
Total params: 131
Trainable params: 131
Non-trainable params: 0
_________________________________________________________________


In [3]:
def input_fn():
    split = tfds.Split.TRAIN
    dataset = tfds.load('iris', split=split, as_supervised=True)
    dataset = dataset.map(lambda features, labels: ({'dense_input':features}, labels))
    dataset = dataset.batch(32).repeat()

    return dataset

# 4.通过 tf.keras 模型创建 Estimator

- 使用 tf.estimator API 来训练 tf.keras.Model，方法是使用 tf.keras.estimator.model_to_estimator 将模型转换为 tf.estimator.Estimator 对象。

In [5]:
model_dir = tempfile.mkdtemp()
keras_estimator = tf.keras.estimator.model_to_estimator(
    keras_model = model, 
    model_dir = model_dir,
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using the Keras model provided.


/Users/zfwang/.pyenv/versions/3.7.10/envs/tf2/lib/python3.7/site-packages/tensorflow/python/keras/backend.py:434: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


INFO:tensorflow:Using config: {'_model_dir': '/var/folders/mg/50rhj31j7m955hzvnjqw0w1m0000gn/T/tmprktxqjkg', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [ ]:
keras_estimator.train(input_fn = input_fn, steps = 500)
eval_result = keras_estimator.evaluate(input_fn = input_fn, steps = 10)
print(f"Eval result: {eval_result}")

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
